In [126]:
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [127]:
maze_data_loc = "Statistics/metric.csv"

maze_colors = {
    'Aldous Broder': '#1f77b4',   
    'Hunt and Kill': '#ff7f0e',     
    'Loop Hunt and Kill': '#c49c94',  
    'Initialized Prim\'s': '#2ca02c', 
    'Depth First Prim\'s': '#d62728',        
    'Randomized Kruskal\'s': '#9467bd',
    'Sidewinder': '#8c564b',      
    'Stochastic Prim\'s': '#e377c2',  
    'Wilson\'s': '#7f7f7f',          
    'Randomized Prim\'s': '#bcbd22',
    'Loop Prim': '#17becf',
    'Recursive Backtracker': '#aec7e8',
    'Randomized DFS': '#ff9896'
}

metric_df = pd.read_csv(maze_data_loc)

In [128]:
loop_data_loc = "Statistics/loop_metric.csv"

loop_df = pd.read_csv(loop_data_loc)
randomized_prim_df = metric_df[metric_df['Maze Type'] == 'Randomized Prim\'s'].copy()
randomized_prim_df["Maze Type"] = "Loop Prim"
randomized_prim_df["Loop Density"] = 0

hunt_and_kill_df = metric_df[metric_df['Maze Type'] == 'Hunt and Kill'].copy()
hunt_and_kill_df["Maze Type"] = "Loop Hunt and Kill"
hunt_and_kill_df["Loop Density"] = 0

loop_df = pd.concat([loop_df, randomized_prim_df, hunt_and_kill_df], ignore_index=True)


In [129]:
solution_data_loc = "Statistics/solution.csv"

solver_colors = {
    'A*': '#1f77b4',
    'BFS': '#ff7f0e',
    'DFS': '#2ca02c',
    'HandOnWall': '#d62728',
    'HybridBFSDFS': '#9467bd',
    'RandomWalk': '#8c564b'}

solution_df = pd.read_csv(solution_data_loc)


In [130]:
loop_solution_data_loc = "Statistics/loop_solution.csv"

loop_solution_df = pd.read_csv(loop_solution_data_loc)

randomized_prim_solution_df = solution_df[solution_df['Maze Type'] == 'Randomized Prim\'s'].copy()
randomized_prim_solution_df["Maze Type"] = "Loop Prim"
randomized_prim_solution_df["Loop Density"] = 0
randomized_prim_solution_df = randomized_prim_solution_df[randomized_prim_solution_df["Solver"] != "HandOnWall"]

hunt_and_kill_solution_df = solution_df[solution_df['Maze Type'] == 'Hunt and Kill'].copy()
hunt_and_kill_solution_df["Maze Type"] = "Loop Hunt and Kill"
hunt_and_kill_solution_df["Loop Density"] = 0
hunt_and_kill_solution_df = hunt_and_kill_solution_df[hunt_and_kill_solution_df["Solver"] != "HandOnWall"]

loop_solution_df = pd.concat([loop_solution_df, randomized_prim_solution_df, hunt_and_kill_solution_df], ignore_index=True)



In [131]:
dqn_data_loc = "Statistics/DQN_experiment_results.csv"

dqn_df = pd.read_csv(dqn_data_loc)



In [132]:
def box_plot_metric(column):
    plt.figure(figsize=(12, 6))  

    sns.boxplot(
        data=metric_df, x='Maze Type', y=column, hue='Maze Type',
        palette=maze_colors, legend=False
    )
               
    plt.title(f'{column}', fontsize=12, fontweight='bold')
    plt.xticks(rotation=0, ha='center', fontsize=7)
    plt.ylabel('')
    plt.xlabel('')
    plt.tight_layout()

    plt.show()

In [133]:
def box_plot_solution(column, maze_type=None):
    plt.figure(figsize=(12, 6))  
        
    if maze_type is not None:
        plt.suptitle(f"Solver Performance on {maze_type}", fontsize=15)
        df_filtered = solution_df[solution_df['Maze Type'] == maze_type]
    else:
        df_filtered = solution_df
        plt.suptitle("Solver Performance on All Mazes", fontsize=15)
        
    sns.boxplot(
        data=df_filtered, x='Solver', y=column, hue='Solver', 
        palette=solver_colors, legend=False
    )
    plt.title(f'{column}', fontsize=12, fontweight='bold')
    plt.xticks(rotation=0, ha='center', fontsize=7)
    plt.ylabel('')
    plt.xlabel('')
    plt.tight_layout()

    plt.show()

In [134]:
def box_plot_loop_solution(column, maze_type=None):
    plt.figure(figsize=(12, 6))  
        
    if maze_type is not None:
        plt.suptitle(f"Solver Performance on {maze_type}", fontsize=15)
        df_filtered = loop_solution_df[loop_solution_df['Maze Type'] == maze_type]
    else:
        df_filtered = loop_solution_df
        plt.suptitle("Solver Performance on All Mazes", fontsize=15)
        
    sns.boxplot(
        data=df_filtered, x='Solver', y=column, hue='Solver', 
        palette=solver_colors, legend=False
    )
    plt.title(f'{column}', fontsize=12, fontweight='bold')
    plt.xticks(rotation=0, ha='center', fontsize=7)
    plt.ylabel('')
    plt.xlabel('')
    plt.tight_layout()

    plt.show()

In [135]:
def pivot_tables_solution(column):
    pivot_df = solution_df.pivot_table(
        index='Maze Type',
        columns='Solver',
        values=column,
        aggfunc='mean'  
    )

    pivot_df['Mean'] = pivot_df.mean(axis=1)
    pivot_df = pivot_df.sort_values(by='Mean', ascending=False)

    pivot_df = pivot_df.drop(columns='Mean')

    print(f"Comparison Table of {column} for each Solver:\n")
    return pivot_df

In [136]:
def pivot_tables_loop_solution(column):
    pivot_df = loop_solution_df.pivot_table(
        index=['Maze Type', 'Loop Density'],
        columns='Solver',
        values=column,
        aggfunc='mean'
    )

    pivot_df = pivot_df.sort_index(level=['Maze Type', 'Loop Density'], ascending=[True, True])

    print(f"Comparison Table of {column} for each Solver:\n")
    return pivot_df

In [137]:
def ranking_table_solution(solver):
    solver_df = solution_df[solution_df['Solver'] == solver]

    avg_df = solver_df.groupby("Maze Type").mean(numeric_only=True)

    avg_df = avg_df.drop(columns=["Visited Dead Ends", "Visited Intersections"])

    ascending = {
    "Visited Nodes": True,
    "Solution Path Length": True,
    "Excess Steps": True,
    "Exploration Ratio": True,
    "Visited Dead Ends": True,
    "Visited Intersections": True,
    "Dead End Coverage": True,
    "Intersection Coverage": True
    } 
    
    rank_df = avg_df.rank(
    axis=0,
    method='min',
    ascending=[ascending.get(col, True) for col in avg_df.columns]
    ).astype(int)

    rank_df= rank_df.sort_values(by="Excess Steps")

    rank_df = rank_df.drop(columns="Exploration Ratio")

    print(f"Ranked metrics for solver '{solver}':\n")
    return rank_df

In [138]:
def average_table_solution(solver):
    solver_df = solution_df[solution_df['Solver'] == solver]

    avg_df = solver_df.groupby("Maze Type").mean(numeric_only=True)

    avg_df = avg_df.drop(columns=["Visited Dead Ends", "Visited Intersections"])

    avg_df = avg_df.sort_values(by="Excess Steps")

    avg_df = avg_df.drop(columns="Exploration Ratio")

    print(f"Average metrics for solver '{solver}':\n")
    return avg_df

In [139]:
def ranking_table_loop_solution(solver):
    solver_df = loop_solution_df[loop_solution_df['Solver'] == solver]

    avg_df = solver_df.groupby(["Maze Type", "Loop Density"]).mean(numeric_only=True)

    ascending = {
        "Visited Nodes": True,
        "Solution Path Length": True,
        "Excess Steps": True,
        "Exploration Ratio": True,
        "Visited Dead Ends": True,
        "Visited Intersections": True,
        "Dead End Coverage": True,
        "Intersection Coverage": True
    }

    def rank_group(df):
        ranks = df.rank(
            axis=0,
            method='min',
            ascending=[ascending.get(col, True) for col in df.columns]
        )
        return ranks.astype(int)

    rank_df = avg_df.groupby(level=0, group_keys=False).apply(rank_group)

    rank_df["Average Rank"] = rank_df.mean(axis=1)
    rank_df = rank_df.sort_index(level=['Maze Type', 'Loop Density'])

    rank_df = rank_df.drop(columns="Average Rank")
    print(f"Ranked metrics for solver '{solver}' (grouped by Maze Type):\n")
    return rank_df


In [140]:
metric_avg_df = metric_df.groupby("Maze Type").mean(numeric_only=True)
metric_avg_df = metric_avg_df.round(2)

metric_avg_df.drop(columns = ["Max Distance to Deadend(From Solution Path)", "Avg Distance to Deadend(From Intersection)", "Shortest Possible Solution"], inplace = True)
metric_avg_df= metric_avg_df.sort_values(by = "Avg Distance to Deadend(From Solution Path)")

metric_avg_df

,# of Intersections,Solution Length,Solution Path Tortuosity,# of Deadends,Deadend Crossroads,Avg Distance to Deadend(From Solution Path),Tempting Count
Maze Type,,,,,,,
Depth First Prim's,198.27,51.10,1.06,248.31,171.82,31.37,45.75
Randomized Prim's,168.94,51.32,1.07,198.21,113.49,32.61,38.05
Randomized Kruskal's,163.39,75.16,1.57,187.85,107.12,38.09,44.78
Stochastic Prim's,174.94,48.02,1.00,196.78,112.74,38.15,25.69
Sidewinder,153.60,59.36,1.24,171.00,90.54,39.06,30.59
Wilson's,160.10,77.88,1.62,182.68,103.80,41.24,43.72
Aldous Broder,157.97,81.10,1.69,179.53,102.21,42.33,44.71
Hunt and Kill,58.39,107.10,2.23,60.21,18.15,69.39,16.38
Randomized DFS,61.86,230.80,4.81,63.47,31.85,137.43,17.61


## Solver Results

In [152]:
average_table_solution("BFS")

Average metrics for solver 'BFS':



,Visited Nodes,Solution Path Length,Excess Steps,Dead End Coverage,Intersection Coverage
Maze Type,,,,,
Randomized DFS,358.59,217.06,141.53,0.4590,0.5019
Recursive Backtracker,376.91,227.62,149.29,0.4904,0.5309
Wilson's,505.85,75.54,430.31,0.7889,0.8289
Aldous Broder,514.22,77.84,436.38,0.8080,0.8436
Hunt and Kill,555.12,102.64,452.48,0.8579,0.9160
Randomized Kruskal's,532.35,74.30,458.05,0.8338,0.8738
Stochastic Prim's,557.97,48.12,509.85,0.8601,0.9248
Sidewinder,597.28,59.60,537.68,0.9384,0.9673
Depth First Prim's,619.39,51.18,568.21,0.9885,0.9976


In [141]:
ranking_table_solution("BFS")

Ranked metrics for solver 'BFS':



,Visited Nodes,Solution Path Length,Excess Steps,Dead End Coverage,Intersection Coverage
Maze Type,,,,,
Randomized DFS,1,9,1,1,1
Recursive Backtracker,2,10,2,2,2
Wilson's,3,6,3,3,3
Aldous Broder,4,7,4,4,4
Hunt and Kill,6,8,5,6,6
Randomized Kruskal's,5,5,6,5,5
Stochastic Prim's,7,1,7,7,7
Sidewinder,8,4,8,8,8
Depth First Prim's,9,2,9,9,9


In [151]:
average_table_solution("DFS")

Average metrics for solver 'DFS':



,Visited Nodes,Solution Path Length,Excess Steps,Dead End Coverage,Intersection Coverage
Maze Type,,,,,
Recursive Backtracker,385.83,227.62,158.21,0.4313,0.5675
Randomized DFS,394.21,217.06,177.15,0.4590,0.5850
Hunt and Kill,318.09,102.64,215.45,0.4721,0.5606
Randomized Kruskal's,363.98,74.30,289.68,0.5284,0.6371
Randomized Prim's,348.95,51.70,297.25,0.5324,0.5969
Depth First Prim's,349.19,51.18,298.01,0.5429,0.5799
Aldous Broder,417.38,77.84,339.54,0.6404,0.6965
Wilson's,429.17,75.54,353.63,0.6786,0.6954
Sidewinder,466.85,59.60,407.25,0.7096,0.7873


In [142]:
ranking_table_solution("DFS")

Ranked metrics for solver 'DFS':



,Visited Nodes,Solution Path Length,Excess Steps,Dead End Coverage,Intersection Coverage
Maze Type,,,,,
Recursive Backtracker,5,10,1,1,2
Randomized DFS,6,9,2,2,4
Hunt and Kill,1,8,3,3,1
Randomized Kruskal's,4,5,4,4,6
Randomized Prim's,2,3,5,5,5
Depth First Prim's,3,2,6,6,3
Aldous Broder,7,7,7,7,8
Wilson's,8,6,8,8,7
Sidewinder,10,4,9,9,10


In [150]:
average_table_solution("A*")

Average metrics for solver 'A*':



,Visited Nodes,Solution Path Length,Excess Steps,Dead End Coverage,Intersection Coverage
Maze Type,,,,,
Randomized DFS,328.75,217.06,111.69,0.3958,0.4473
Recursive Backtracker,350.81,227.62,123.19,0.4309,0.4859
Stochastic Prim's,225.07,48.12,176.95,0.2126,0.4928
Depth First Prim's,341.44,51.18,290.26,0.4475,0.6519
Wilson's,366.64,75.54,291.10,0.5405,0.6250
Randomized Kruskal's,365.41,74.30,291.11,0.5397,0.6264
Aldous Broder,376.94,77.84,299.10,0.5620,0.6404
Randomized Prim's,395.60,51.70,343.90,0.5595,0.7141
Sidewinder,415.05,59.60,355.45,0.6189,0.7012


In [143]:
ranking_table_solution("A*")

Ranked metrics for solver 'A*':



,Visited Nodes,Solution Path Length,Excess Steps,Dead End Coverage,Intersection Coverage
Maze Type,,,,,
Randomized DFS,2,9,1,2,1
Recursive Backtracker,4,10,2,3,2
Stochastic Prim's,1,1,3,1,3
Depth First Prim's,3,2,4,4,7
Wilson's,6,6,5,6,4
Randomized Kruskal's,5,5,6,5,5
Aldous Broder,7,7,7,8,6
Randomized Prim's,8,3,8,7,9
Sidewinder,9,4,9,9,8


In [149]:
average_table_solution("HybridBFSDFS")

Average metrics for solver 'HybridBFSDFS':



,Visited Nodes,Solution Path Length,Excess Steps,Dead End Coverage,Intersection Coverage
Maze Type,,,,,
Randomized DFS,300.26,217.06,83.20,0.2927,0.4018
Recursive Backtracker,321.06,227.62,93.44,0.3154,0.4330
Wilson's,367.20,75.54,291.66,0.5459,0.6267
Randomized Kruskal's,375.19,74.30,300.89,0.5518,0.6482
Aldous Broder,384.58,77.84,306.74,0.5675,0.6606
Hunt and Kill,416.80,102.64,314.16,0.5975,0.7355
Stochastic Prim's,369.50,48.12,321.38,0.5285,0.6472
Sidewinder,438.03,59.60,378.43,0.6531,0.7489
Randomized Prim's,436.00,51.70,384.30,0.6555,0.7458


In [144]:
ranking_table_solution("HybridBFSDFS")

Ranked metrics for solver 'HybridBFSDFS':



,Visited Nodes,Solution Path Length,Excess Steps,Dead End Coverage,Intersection Coverage
Maze Type,,,,,
Randomized DFS,1,9,1,1,1
Recursive Backtracker,2,10,2,2,2
Wilson's,3,6,3,4,3
Randomized Kruskal's,5,5,4,5,5
Aldous Broder,6,7,5,6,6
Hunt and Kill,7,8,6,7,7
Stochastic Prim's,4,1,7,3,4
Sidewinder,9,4,8,8,9
Randomized Prim's,8,3,9,9,8


In [148]:
average_table_solution("RandomWalk")

Average metrics for solver 'RandomWalk':



,Visited Nodes,Solution Path Length,Excess Steps,Dead End Coverage,Intersection Coverage
Maze Type,,,,,
Randomized DFS,397.18,217.06,180.12,0.4521,0.5877
Recursive Backtracker,412.11,227.62,184.49,0.4760,0.6158
Hunt and Kill,353.85,102.64,251.21,0.4803,0.6150
Wilson's,344.92,75.54,269.38,0.4918,0.6080
Depth First Prim's,320.61,51.18,269.43,0.4742,0.5620
Sidewinder,335.46,59.60,275.86,0.4877,0.5787
Aldous Broder,357.18,77.84,279.34,0.5053,0.6271
Randomized Prim's,338.72,51.70,287.02,0.5026,0.5934
Stochastic Prim's,337.01,48.12,288.89,0.5021,0.5763


In [145]:
ranking_table_solution("RandomWalk")

Ranked metrics for solver 'RandomWalk':



,Visited Nodes,Solution Path Length,Excess Steps,Dead End Coverage,Intersection Coverage
Maze Type,,,,,
Randomized DFS,9,9,1,1,4
Recursive Backtracker,10,10,2,3,8
Hunt and Kill,6,8,3,4,7
Wilson's,5,6,4,6,6
Depth First Prim's,1,2,5,2,1
Sidewinder,2,4,6,5,3
Aldous Broder,7,7,7,9,9
Randomized Prim's,4,3,8,8,5
Stochastic Prim's,3,1,9,7,2


In [146]:
average_table_solution("HandOnWall")

Average metrics for solver 'HandOnWall':



,Visited Nodes,Solution Path Length,Excess Steps,Dead End Coverage,Intersection Coverage
Maze Type,,,,,
Sidewinder,142.08,59.60,82.48,0.1527,0.2791
Hunt and Kill,252.05,102.64,149.41,0.3163,0.4727
Recursive Backtracker,398.92,227.62,171.30,0.4547,0.5949
Randomized DFS,403.62,217.06,186.56,0.4693,0.6040
Stochastic Prim's,300.02,48.12,251.90,0.4383,0.5219
Aldous Broder,343.29,77.84,265.45,0.4860,0.6062
Wilson's,349.30,75.54,273.76,0.4994,0.6156
Randomized Kruskal's,352.78,74.30,278.48,0.5055,0.6221
Depth First Prim's,331.63,51.18,280.45,0.4903,0.5791


In [147]:
ranking_table_solution("HandOnWall")

Ranked metrics for solver 'HandOnWall':



,Visited Nodes,Solution Path Length,Excess Steps,Dead End Coverage,Intersection Coverage
Maze Type,,,,,
Sidewinder,1,4,1,1,1
Hunt and Kill,2,8,2,2,2
Recursive Backtracker,9,10,3,4,5
Randomized DFS,10,9,4,5,7
Stochastic Prim's,3,1,5,3,3
Aldous Broder,6,7,6,6,8
Wilson's,7,6,7,8,9
Randomized Kruskal's,8,5,8,9,10
Depth First Prim's,4,2,9,7,4
